In [1]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

2021-08-10 10:58:40.310409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2021-08-10 10:58:40.310438: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import sys
sys.modules[__name__]
sys.path.append("/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src")
sys.path.append("/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages")
# sys.path.append("/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src/moveo_training/scripts")
print(os.getcwd())
print(sys.executable)
print(sys.path)

/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL
/home/nils/anaconda3/bin/python
['/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL', '', '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/home/nils/anaconda3/lib/python38.zip', '/home/nils/anaconda3/lib/python3.8', '/home/nils/anaconda3/lib/python3.8/lib-dynload', '/home/nils/.local/lib/python3.8/site-packages', '/home/nils/anaconda3/lib/python3.8/site-packages', '/home/nils/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg', '/home/nils/.local/lib/python3.8/site-packages/IPython/extensions', '/home/nils/.ipython', '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src', '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages']


In [3]:
import tensorflow as tf

from tf_agents.environments import suite_gym
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils
import rospy
import tempfile

import moveo_training.scripts.moveo_inverse_kinematic

tempdir = tempfile.gettempdir()

Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [4]:
rospy.init_node("train_moveo_her")
env_name = 'MoveoIK-v0' # @param {type:"string"}


In [5]:
# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 1 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

In [8]:

import gym
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env_name in env:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]



Remove MoveoIK-v0 from registry


In [12]:
from gym.envs.registration import register
register(
        id='MoveoIK-v0',
        entry_point='moveo_training.scripts.moveo_inverse_kinematic:MoveoIKEnv',
        max_episode_steps=100,
    )


Error: Cannot re-register id: MoveoIK-v0

In [15]:
env = suite_gym.load(env_name)
env.reset()

###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1628586380.235068, 1822.802000]: NOT Initialising Simulation Physics Parameters
[WARN] [1628586380.241364, 1822.808000]: Start Init ControllersConnection
[WARN] [1628586380.242596, 1822.808000]: END Init ControllersConnection


AssertionError: Initialisation is failed....